# **Drug Consumption Classification**

## **Imports**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('data/drug_consumption.csv')
data1 = pd.read_csv('data/drug_consumption.csv')
clean_dataset = dataset.copy()

In [3]:
# drop id column
dataset = dataset.drop('ID', axis=1)

In [4]:
dataset.head(10)

,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,Cscore,...,Ecstasy,Heroin,Ketamine,Legalh,LSD,Meth,Mushrooms,Nicotine,Semer,VSA
0,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
1,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,...,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
2,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,...,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
3,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,...,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
4,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,...,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0
5,2.59171,0.48246,-1.22751,0.24923,-0.31685,-0.67825,-0.30033,-1.55521,2.03972,1.63088,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
6,1.09449,-0.48246,1.16365,-0.57009,-0.31685,-0.46725,-1.09207,-0.45174,-0.30172,0.93949,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
7,0.49788,-0.48246,-1.73790,0.96082,-0.31685,-1.32828,1.93886,-0.84732,-0.30172,1.63088,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0
8,0.49788,0.48246,-0.05921,0.24923,-0.31685,0.62967,2.57309,-0.97631,0.76096,1.13407,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
9,1.82213,-0.48246,1.16365,0.96082,-0.31685,-0.24649,0.00332,-1.42424,0.59042,0.12331,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0


In [5]:
dataset.describe()

,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SS
count,1885.00000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000,1885.000000
mean,0.03461,-0.000256,-0.003806,0.355542,-0.309577,0.000047,-0.000163,-0.000534,-0.000245,-0.000386,0.007216,-0.003292
std,0.87836,0.482588,0.950078,0.700335,0.166226,0.998106,0.997448,0.996229,0.997440,0.997523,0.954435,0.963701
min,-0.95197,-0.482460,-2.435910,-0.570090,-1.107020,-3.464360,-3.273930,-3.273930,-3.464360,-3.464360,-2.555240,-2.078480
25%,-0.95197,-0.482460,-0.611130,-0.570090,-0.316850,-0.678250,-0.695090,-0.717270,-0.606330,-0.652530,-0.711260,-0.525930
50%,-0.07854,-0.482460,-0.059210,0.960820,-0.316850,0.042570,0.003320,-0.019280,-0.017290,-0.006650,-0.217120,0.079870
75%,0.49788,0.482460,0.454680,0.960820,-0.316850,0.629670,0.637790,0.723300,0.760960,0.584890,0.529750,0.765400
max,2.59171,0.482460,1.984370,0.960820,1.907250,3.273930,3.273930,2.901610,3.464360,3.464360,2.901610,1.921730


In [6]:
age_col = {
    -0.95197: '18-24',
    -0.07854: '25 - 34',
    0.49788: '35 - 44',
    1.09449: '45 - 54',
    1.82213: '55 - 64',
    2.59171: '65+'
}
dataset['Age'] = dataset['Age'].replace(age_col)

gender_col = {
    0.48246: 'Female',
    -0.48246: 'Male'
}
dataset['Gender'] = dataset['Gender'].replace(gender_col)

education_col = {
    -2.43591: 'Left School Before 16 years',
    -1.73790: 'Left School at 16 years',
    -1.43719: 'Left School at 17 years',
    -1.22751: 'Left School at 18 years',
    -0.61113: 'Some College,No Certificate Or Degree',
    -0.05921: 'Professional Certificate/ Diploma',
    0.45468: 'University Degree',
    1.16365: 'Masters Degree',
    1.98437: 'Doctorate Degree',
}
dataset['Education'] = dataset['Education'].replace(education_col)

country_col = {
    -0.09765: 'Australia',
    0.24923: 'Canada',
    -0.46841: 'New Zealand',
    -0.28519: 'Other',
    0.21128: 'Republic of Ireland',
    0.96082: 'UK',
    -0.57009: 'USA'
}
dataset['Country'] = dataset['Country'].replace(country_col)

ethnicity_col = {
    -0.50212: 'Asian',
    -1.10702: 'Black',
    1.90725: 'Mixed-Black/Asian',
    0.12600: 'Mixed-White/Asian',
    -0.22166: 'Mixed-White/Black',
    0.11440: 'Other',
    -0.31685: 'White'
}

dataset['Ethnicity'] = dataset['Ethnicity'].replace(ethnicity_col)

usage_col = {
    'CL0': 'Never Used',
    'CL1': 'Used over a Decade Ago',
    'CL2': 'Used in Last Decade',
    'CL3': 'Used in Last Year',
    'CL4': 'Used in Last Month',
    'CL5': 'Used in Last Week',
    'CL6': 'Used in Last Day',
}

dataset['Alcohol'] = dataset['Alcohol'].replace(usage_col)
dataset['Amphet'] = dataset['Amphet'].replace(usage_col)
dataset['Amyl'] = dataset['Amyl'].replace(usage_col)
dataset['Benzos'] = dataset['Benzos'].replace(usage_col)
dataset['Caff'] = dataset['Caff'].replace(usage_col)
dataset['Cannabis'] = dataset['Cannabis'].replace(usage_col)
dataset['Choc'] = dataset['Choc'].replace(usage_col)
dataset['Coke'] = dataset['Coke'].replace(usage_col)
dataset['Crack'] = dataset['Crack'].replace(usage_col)
dataset['Ecstasy'] = dataset['Ecstasy'].replace(usage_col)
dataset['Heroin'] = dataset['Heroin'].replace(usage_col)
dataset['Ketamine'] = dataset['Ketamine'].replace(usage_col)
dataset['Legalh'] = dataset['Legalh'].replace(usage_col)
dataset['LSD'] = dataset['LSD'].replace(usage_col)
dataset['Meth'] = dataset['Meth'].replace(usage_col)
dataset['Mushrooms'] = dataset['Mushrooms'].replace(usage_col)
dataset['Nicotine'] = dataset['Nicotine'].replace(usage_col)
dataset['Semer'] = dataset['Semer'].replace(usage_col)
dataset['VSA'] = dataset['VSA'].replace(usage_col)

In [7]:
dataset.head(10)

,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,Cscore,...,Ecstasy,Heroin,Ketamine,Legalh,LSD,Meth,Mushrooms,Nicotine,Semer,VSA
0,35 - 44,Female,Professional Certificate/ Diploma,UK,Mixed-White/Asian,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,...,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Used in Last Decade,Never Used,Never Used
1,25 - 34,Male,Doctorate Degree,UK,White,-0.67825,1.93886,1.43533,0.76096,-0.14277,...,Used in Last Month,Never Used,Used in Last Decade,Never Used,Used in Last Decade,Used in Last Year,Never Used,Used in Last Month,Never Used,Never Used
2,35 - 44,Male,Professional Certificate/ Diploma,UK,White,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,...,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Used over a Decade Ago,Never Used,Never Used,Never Used
3,18-24,Female,Masters Degree,UK,White,-0.14882,-0.80615,-0.01928,0.59042,0.58489,...,Never Used,Never Used,Used in Last Decade,Never Used,Never Used,Never Used,Never Used,Used in Last Decade,Never Used,Never Used
4,35 - 44,Female,Doctorate Degree,UK,White,0.73545,-1.63340,-0.45174,-0.30172,1.30612,...,Used over a Decade Ago,Never Used,Never Used,Used over a Decade Ago,Never Used,Never Used,Used in Last Decade,Used in Last Decade,Never Used,Never Used
5,65+,Female,Left School at 18 years,Canada,White,-0.67825,-0.30033,-1.55521,2.03972,1.63088,...,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Used in Last Day,Never Used,Never Used
6,45 - 54,Male,Masters Degree,USA,White,-0.46725,-1.09207,-0.45174,-0.30172,0.93949,...,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Used in Last Day,Never Used,Never Used
7,35 - 44,Male,Left School at 16 years,UK,White,-1.32828,1.93886,-0.84732,-0.30172,1.63088,...,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used
8,35 - 44,Female,Professional Certificate/ Diploma,Canada,White,0.62967,2.57309,-0.97631,0.76096,1.13407,...,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Used in Last Day,Never Used,Never Used
9,55 - 64,Male,Masters Degree,UK,White,-0.24649,0.00332,-1.42424,0.59042,0.12331,...,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Used in Last Day,Never Used,Never Used


In [ ]:
# show all unique values in each column
for col in dataset.columns:
    print(f"{col}: {dataset[col].unique()}")

In [ ]:
# draw bar plots that count the number of rows for each unique value in each column
for col in dataset.columns:
    if col in ['Cannabis']:
        plt.figure(figsize=(10, 6))
        dataset[col].value_counts().plot(kind='bar')
        plt.title(f'{col} Distribution')
        plt.xlabel(col)
        plt.ylabel('Count')
        plt.show()

    


In [15]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Assuming 'X' is your dataset and 'nominal_columns' is a list of column names to be encoded
nominal_columns = ['Age', 'Gender', 'Education', 'Country', 'Ethnicity']
# One-Hot Encoding
onehot_encoder = OneHotEncoder(sparse_output=False)
onehot_encoded = onehot_encoder.fit_transform(dataset[nominal_columns])

# Label Encoding
label_encoder = LabelEncoder()
label_encoded = dataset[nominal_columns].apply(label_encoder.fit_transform)


In [16]:
label_encoded

,Age,Gender,Education,Country,Ethnicity
0,2,0,6,5,3
1,1,1,0,5,6
2,2,1,6,5,6
3,0,0,5,5,6
4,2,0,0,5,6
...,...,...,...,...,...
1880,0,0,7,6,6
1881,0,1,7,6,6
1882,1,0,8,6,6
1883,0,0,7,6,6


In [17]:
onehot_encoded


array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.]])

In [ ]:
# # create new datasets for each unique value in the 'Cannabis' column

# cannabis_datasets = []
# prediction_columns = ['Age', 'Gender', 'Education', 'Country', 'Ethnicity', 'Nscore', 'Escore', 'Oscore', 'Ascore', 'Cscore', 'Impulsive', 'SS']

# for value in dataset['Cannabis'].unique():
#     # filter the dataset to only include columns in the 'prediction_columns' list
#     cannabis_dataset = dataset[dataset['Cannabis'] == value][prediction_columns]
#     cannabis_datasets.append(cannabis_dataset)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split

In [ ]:
# apply random forest classifier to the dataset
def apply_random_forest_classifier(dataset, target_column):
    # split the dataset into a training set and a test set
    X = dataset.drop(target_column, axis=1)
    y = dataset[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # create a random forest classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)

    # train the classifier
    clf.fit(X_train, y_train)

    # make predictions
    y_pred = clf.predict(X_test)

    # calculate the accuracy of the classifier
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy}')

    # calculate the precision of the classifier
    precision = precision_score(y_test, y_pred, average='weighted')
    print(f'Precision: {precision}')

    # calculate the recall of the classifier
    recall = recall_score(y_test, y_pred, average='weighted')
    print(f'Recall: {recall}')

    # calculate the confusion matrix of the classifier
    cm = confusion_matrix(y_test, y_pred)
    print(f'Confusion Matrix: {cm}')

    # plot the confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
    disp.plot()

    return clf

In [ ]:
# replace the CL0, CL1, CL2, CL3, CL4, CL5, CL6 values in the 'Cannabis' column with 0, 1, 2, 3, 4, 5, 6
cannabis_col = {
    'CL0': 0,
    'CL1': 1,
    'CL2': 2,
    'CL3': 3,
    'CL4': 4,
    'CL5': 5,
    'CL6': 6,
}

data1 = data1.replace(cannabis_col)

In [ ]:
# call the function with the dataset and the target column as 'Cannabis'
clf = apply_random_forest_classifier(data1, 'Cannabis')